# **Agricultural drought in central South America - preparing model data for analysis**

In [1]:
import sys; sys.path.append('/home/clair/wwa'); from wwa import *; from wwa_indices import get_spi

---
## **Pre-processing**
Time series, spatial patterns & seasonal cycle all extracted from data downloaded using synda

---
### **CORDEX**

#### **Seasonal cycle**


#### **Spatial pattern**
Using CPC as reference - agrees very closely with MSWEP, also gives temperature data if looking at SPEI
